# Pandas GroupBy

One of the most powerful features of Pandas is its **GroupBy** functionality. Sometimes we want to perform operations on *groups* of observations within a dataset. For example:

* **aggregation**, such as computing the sum of mean of each group, which involves applying a function to each group and returning the aggregated results
* **slicing** the DataFrame into groups and then doing something with the resulting slices (*e.g.* plotting)
* group-wise **transformation**, such as standardization/normalization

### Some MLB Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Let's start with an example. We have some data about Major League Baseball salaries in a `csv` file.

In [ ]:
 mlb = pd.read_csv('mlbsalaries.csv')

In [ ]:
 mlb.head()

In [ ]:
mlb.tail()

In [ ]:
mlb.info()

Our dataframe has 19543 rows. It looks like a tidy dataset with no missing values.

## pandas GroupBy

The `groupby` method in pandas is easy to use. 

Let's create a pandas DataFrameGroupBy object by calling `groupby` on `mlb`. This will "split" the original data into a set of dataframes based on the values in the `Year` column.

In [ ]:
mlb_by_year = mlb.groupby('Year')

This *grouped* dataset is hard to visualize. It has a special pandas type called "DataFrameGroupBy."

Essentially, our grouped object is a set of key-value pairs where the keys are the grouping variable (`year`) and the values are dataframes.

In [ ]:
mlb_by_year

However, the grouping is only an intermediate step.

For example, we may want to iterate over each of the `year` groups:

In [ ]:
 for k,group in mlb_by_year:
        print(k)
        print(group.info())
        print

As expected, we have split the original data into a set of dataframes with one for each year.

## Split-Apply-Combine

pandas `groupby` method draws largely from the [split-apply-combine strategy for data analysis](http://www.jstatsoft.org/v40/i01/paper). The **split-apply-combine** operation groups subsets of data together, applies a function to each of the groups, then recombines them into a new data table. It is a common data analysis procedure.

For example, we may want to aggregate our data with with some function.

![split-apply-combine](http://f.cl.ly/items/0s0Z252j0X0c3k3P1M47/Screen%20Shot%202013-06-02%20at%203.04.04%20PM.png)
<div align="right">*(figure taken from "Python for Data Analysis", p.251)*</div>

We can aggregate in Pandas using the aggregate (or agg, for short) method:

In [ ]:
mlb_by_year.agg(np.median).head()

In [ ]:
mlb_by_year.agg(sum).head()

Some aggregation functions are so common that Pandas has a convenience method for them, such as `median`:

In [ ]:
mlb_by_year.median().head()

We can also get summary information directly from the original dataframe:

In [ ]:
mlb.groupby('Year').median().head()

### More about Split-Apply-Combine

When approaching a data analysis problem, you'll often break it apart into manageable pieces, perform some operations on each of the pieces, and then put everything back together again (this is the gist split-apply-combine strategy). pandas `groupby` is great for these problems.

Assume we have a DataFrame and want to get the average for each group - visually, the split-apply-combine method looks like this:

![Source: Gratuitously borrowed from [Hadley Wickham's Data Science in R slides](http://courses.had.co.nz/12-oscon/)](http://i.imgur.com/yjNkiwL.png)

### Step-by-step

<b>Step1 (Split): </b> The <i>groupby</i> operation <b><i>splits</b></i> the dataframe into a group of dataframe based on some criteria. Note that the grouped object is <i>not</i> a dataframe. It has a dictionary-like structure and is also iterable.

<img src="groupby1.jpg">

<b>Step 2 (Analyze):</b> Once we have a grouped object we can <b><i>apply</b></i> functions or run analysis to each group, set of groups, or the entire group. 

<img src="groupby2.jpg">

<b>Step 3 (Combine):</b> We can also <b><i>combine</b></i> the results of the analysis into a new data structure(s). 

<img src="groupby3.jpg">

### Extract highest-paid player from each year

We'll use a concept called a "generator comprehension" to identify the highest paid player in each group (year). This is somewhat complicated. Don't worry if you don't fully understand this yet. It should be fairly easy to get a rough idea about what's going on.

We want to sort each group by `Salary` in descending order and select the top row in that group.

In [ ]:
# generator comprehension for retrieving highest paid player by year
t = (group.sort_values(by='Salary', ascending=False)[:1] for yr,group in mlb_by_year)

In [ ]:
t

In just one line, we've just run a powerful analysis. This is the "apply" step.

Now let's "combine" the information in a new dataframe.

### Create new dataframe with list of highest-paid players by year

Let's create an empty dataframe. We'll populate it with the top salaries information from the previous step.

In [ ]:
topsalaries = pd.DataFrame()

Extract the information from each line in the "generator object" `t` and append it to the new dataframe.

In [ ]:
for line in t:
    topsalaries = topsalaries.append(line)

In [ ]:
topsalaries.info()

In [ ]:
topsalaries

## Voila!

We *split* the original data into groups based on year. We *applied* a function to get the highest paid player each year. Then we *combined* things into a new dataframe.

## Visualization Break

Let's do some simple visualization of this new dataframe of top salaries. 

Start by reducing it to the columns of interest and then using `Year` as the index.

In [ ]:
ts = topsalaries[['Year','Salary']]

In [ ]:
ts

In [ ]:
ts = ts.set_index('Year')

In [ ]:
ts

Make a simple plot.

In [ ]:
ts.plot();

We can do simple plots directly on the grouped object by applying dataframe methods (calling functions) on it.

In [ ]:
mlb_by_year['Salary'].median().plot();

In fact, we could have done the same "top salaries" plot simply by using the `max` method:

In [ ]:
mlb_by_year['Salary'].max().plot();

Try some others like `min`.

In [ ]:
# plot the minimum salary per year

## Titanic Example

In [ ]:
titanic = pd.read_csv('train.csv')

In [ ]:
titanic.info()

In [ ]:
titanic.describe()

In [ ]:
titanic.Age.hist(bins=20);

In [ ]:
titanic_by_gender = titanic.groupby('Sex')

In [ ]:
titanic_by_gender.Age.median()

In [ ]:
titanic_by_gender.Age.describe()

In [ ]:
titanic_by_gender.get_group('male').Age.hist(bins=20)
titanic_by_gender.get_group('female').Age.hist(bins=20);

In [ ]:
fig = plt.figure(figsize=(20,6))
titanic_by_gender.boxplot('Age', return_type='dict');

Calling `count` returns the total number of NOT NULL values within each column.

In [ ]:
titanic_by_gender.count()

If we we're interested in the total number of records in each group, we can use `size`.

In [ ]:
titanic_by_gender.size()

### More things

We can group by multiple keys.

In [ ]:
titanic.groupby(['Sex','Pclass']).median().head()

We can **transform** the data, using a function of our choice with the `transform` method:

In [ ]:
normalize = lambda x: (x - x.mean())/x.std()

titanic.groupby(['Sex','Pclass']).transform(normalize).head()

If you simply want to divide your DataFrame into chunks for later use, its easy to convert them into a dict so that they can be easily indexed out as needed:

In [ ]:
chunks = dict(list(titanic.groupby(['Sex','Pclass'])))

In [ ]:
chunks

In [ ]:
chunks['female', 1]

This tutorial was created using information gathered from many web sites, tutorials, and Jupyter (iPython) Notebooks. Much of it is original.